# Semáforos

Ya se han visto en la teoría la diferencia entre hilos y procesos. En este notebook vamos a aprender a utilizar semáforos en hilos en lugar de procesos porque simplifica la compartición de variables.

## Funcionalidad

Un semáforo se puede ver como una variable que tiene un valor entero:
* Puede iniciarse con un valor no negativo.
* La operación down (wait) (P según la notación Dijkstra):
    * Si semaforo > 0: disminuye el valor del semáforo
    * Si semaforo = 0: el proceso se duerme.
* La operación up (signal) (V según la notación Dijkstra) incrementa el valor del semáforo.
    * Si hay procesos durmiendo en ese semáforo despierta a alguno de ellos.
    * De lo contrario incrementa el valor del semáforo.

## Generalizando un poco

Vamos a hacer algo un poco más genérico en el que creamos un procedimiento Launcher. En el primer parámetro se almacenan los identificadores de los hilos en forma de lista. Es una función de parámetros variables en el que se pasa un conjunto de hilos a crear e iniciar. Con el fin de iniciar todos los hilos lo más simultáneamente posible se espera a que sean creados todos antes de ser lanzados.

Así mismo vamos a empezar a trabajar con semáforos. Por simplicidad para aprender el uso de semáforos vamos a utilizar los semáforos de hilos para aprender a trabajar con ellos. En el código de la siguiente celda también se dedinen los procedimientos _up_ y _down_ con el fin de que los problemas a resolver tengan un aspecto muy similar al pseudocódigo de los problemas típicos.

In [ ]:
from threading import Thread, Semaphore
from time import sleep
from random import choice,randint
from string import ascii_letters
import os


def Launcher(threads,*funcs):
    """
    Lanzador de tantos hilos como funciones sean pasadas en los parámetros
    """
    for func in funcs:
        t = Thread(target=func)
        threads.append(t)
    for thr in threads:
        thr.start()
        
def down(sem):
    sem.acquire()

def up(sem):
    sem.release()
    
def SemaphoreArray(N,value=1):
    array = []
    for i in range(N):
        array.append(Semaphore(value))
    return array

## Estucturas básicas con semáforos

### Señalamiento
Esta estructura permite asegurar que un bloque de código de un proceso se realiza antes de que otro proceso realice otro bloque de código. En ejemplo la función _A()_ llamada por un proceso tiene que realizarse antes de que el otro proceso llame a la función _B()_.


In [ ]:
sig = Semaphore(0) #Semaforo inicializado a cero

def A1():
    print("\033[31mHilo A antes del señalamiento\033[00m\n",flush=True,end="")
    
def A2():
    print("\033[32mHilo A después del señalamiento\033[00m\n",flush=True,end="")
    
def B1():
    print("\033[35mHilo B antes del señalamiento\033[00m\n",flush=True,end="")
    
def B2():
    print("\033[32mHilo B después del señalamiento\033[00m\n",flush=True,end="")
    
def proc1():
    A1()
    #sleep(3)
    up(sig)
    A2()
    
def proc2():
    B1()
    down(sig)
    B2()
    
th = []
Launcher(th, proc1, proc2)
sleep(10)
print("\n"*3)
th = []
Launcher(th, proc2, proc1)


### Rendevouz
Esta estructura permite asegurar que un proceso ejecute un elemento de código (A1) antes que un segundo proceso ejecute un trozo de código (B2) y que este segundo proceso ejecute un elemento de código (B1) antes de que el primer proceso ejecute otro trozo de código (A2).


In [ ]:
s1 = Semaphore(0) #Semáforos de sincronización inicializados a cero
s2 = Semaphore(0)

def A1():
    print("\033[31mHilo A antes del rendevouz\033[00m\n",flush=True,end="")
    
def A2():
    print("\033[35mHilo A después del rendevouz\033[00m\n",flush=True,end="")
    
def B1():
    print("\033[32mHiloo B antes del rendevouz\033[00m\n",flush=True,end="")
    
def B2():
    print("\033[33mHilo B después del rendevouz\033[00m\n",flush=True,end="")
    
    
def A():
    A1()
    down(s1)
    up(s2)
    A2()
    
def B():
    B1()
    up(s1)
    down(s2)
    B2()
    
# PAra probarlo, como son procesos muy cortos es crítico el orden de lanzamiento

th = []
Launcher(th, A, B)
sleep(3)
print("\n"*3)
th = []
Launcher(th, B, A)


### Exclusión mútua (Mutex)
Esta estructura permite que sólo un proceso esté en la zona de exclusión mútua impidiendo condiciones de carrera sobre algún recurso.


In [ ]:
mutex = Semaphore(1) #Semáforo de exclusión mutua inicializado a 1

x= 0

def proc1():
    global x
    down(mutex)
    for _ in range(5):
        x += 1
    up(mutex)
    
def proc2():
    global x
    down(mutex)
    for _ in range(5):
        x -= 1
    up(mutex)
    
def proc3():
    global x
    down(mutex)
    for _ in range(3):
        x *= 2
    up(mutex)
    
# Probando el mutex
    
th1 = []
th2 = []
th3 = []
th4 = []

# No asegura el orden pero no hay condiciones de carrera. Posibles soluciones son 0, 35 y -35
# Como los procesos son muy cortos es crítico el orden en el que se lanzan

x=0
Launcher(th1, proc1, proc2, proc3)
print("Secuencia: proc1, proc2, proc3 ==> ",x,"\n")
sleep(1)


x=0
Launcher(th2, proc2, proc1, proc3)
print("Secuencia: proc2, proc1, proc3 ==> ",x,"\n")
sleep(1)


x=0
Launcher(th3, proc1, proc3, proc2)
print("Secuencia: proc1, proc3, proc2 ==> ",x,"\n")
sleep(1)

x=0
Launcher(th4, proc2, proc3, proc1)
print("Secuencia: proc2, proc3, proc1 ==> ",x,"\n")
sleep(1)



### Multiplex
Esta estructura es una generalización del mutex permitiendo que accedan al recurso un número limitado de procesos pero mayor que 1.


In [ ]:
N = 4  # Número de hilos que pueden usar el recurso
multiplex = Semaphore(N)
usando = 0
number=0

# Este semáforo mutex es para poder enseñar la cantidad de procesos que están usando el recurso
mutex = Semaphore(1)
mutex_number = Semaphore(1)


def recurso():
    time=randint(1,10)
    sleep(time)
    

def proc():
    global usando
    global number
    
    # Enumerar los hilos, number es una variable global
    down(mutex_number)
    number+=1
    my_number=number
    up(mutex_number)
    
    
    down(multiplex)
    # Como voy a usar una variable para contar tengo que usar un mutex para que no se produzcan
    # condiciones de carrera pero no es parte de la estructura
    down(mutex)
    usando +=1 
    print("\033[31mThread",my_number,"==> Estamos usando el recurso ", usando, " threads\033[00m\n",flush=True,end="")
    up(mutex)
    
    recurso() # Este es realmente el recurso al que solo pueden acceder N procesos
    
    down(mutex)
    usando -=1 #
    up(mutex)
    
    print("\033[32mThread"+str(my_number)+ "==> Estamos usando el recurso ", usando, " threads\033[00m\n",flush=True,end="")
    up(multiplex)
    
    
th = []

Launcher(th, proc, proc, proc, proc, proc, proc, proc, proc, proc, proc)


### Barreras de un solo uso
Esta estructura es una generalización de la estructura del rendevouz. Esta estructura es válida para maś de dos procesos. La barrera permite esperar a que los procesos esperen hasta que N procesos hayan llegado a la barrera. Esta barrera no es reusable de tal forma que una vez que se levanta la barrera ya pueden pasar todos los procesos que lleguen después de los N primeros.


In [ ]:
mutex = Semaphore(1)
barrier = Semaphore(0)
count = 0

BARRIER_SIZE = 10

def barrier_func():
    global count
    
    down(mutex)
    count += 1
    if count == BARRIER_SIZE:
        up(barrier)
    up (mutex)
    
    down(barrier)
    up(barrier) # El ultimo dejo la barrera abierta
    
    
# Prueba de la barrera

mutexp = Semaphore(1)
num_proc = 0

def probe():
    global num_proc
    # Este código es necesario para que se cojan número para poder identificarlos pero no es parte del problema
    down(mutexp)
    num_proc += 1
    mynum = num_proc
    up(mutexp)
    
    print("\033[31mSoy el Thread "+ str(mynum) +" antes de la barrera\033[00m\n",flush=True,end="")
    barrier_func()
    print("\033[32mSoy el Thread "+ str(mynum) +" después de la barrera\033[00m\n",flush=True,end="")
        
th1 = []
th2 = []
th3 = []

Launcher(th1, probe, probe, probe, probe, probe, probe, probe, probe, probe)
print("Sólo 9 han llegado a la barrera y voy a lanzar el que completa la barrera\n",flush=True,end="")
sleep(4)
Launcher(th2, probe)
print("Voy a lanzar más procesos que pasarán la barrera sin control porque la barrera está levantada\n",flush=True,end="")
Launcher(th3, probe, probe, probe, probe, probe, probe, probe, probe, probe, probe, probe, probe, probe)


### Barreras reusables

La barrera permite esperar a que los procesos esperen hasta que N procesos hayan llegado a la barrera.
Una barrera reusable es aquella que se restaura a si misma para poder volver a ser usada cuando lleguen otros N procesos.

In [ ]:
mutex = Semaphore(1)
mutex_number=Semaphore(1)
reusable_barrier = Semaphore(0)

BARRIER_SIZE = 10
count_in = 0
count_out = 0
number=0

def reusable_barrier_func():
    global count_in, count_out
    
    down(mutex)
    count_in += 1
    if count_in == BARRIER_SIZE:
        print("\033[32mABRIENDO BARRERA PARA UN GRUPO DE "+str(count_in)+"\033[30m\n",flush=True,end="")
        up(barrier)
        count_in = 0
    up (mutex)
    
    down(barrier)
    
    down(mutex)
    count_out += 1
    if count_out !=  BARRIER_SIZE:
        up(barrier)
    else:
        count_out = 0
    up(mutex)
    

    
# Para probarlo

def proc ():
    global number
    
    down(mutex_number)
    number+=1
    my_number=number
    up(mutex_number)

   
    
    print("Entrando Proceso "+str(my_number)+"\n",flush = True,end="")
    reusable_barrier_func()
    sleep(randint(1,5))   
   

        
#Lanzamos 19, sólo pasarán 10
th =[]
funcs1 = [proc] * 19

print("\033[31mLANZANDO GRUPO 19\033[30m\n",flush=True,end="")
Launcher(th,*funcs1)


sleep(5)

#Lanzamos 1 más y como había 9 esperando ...
th =[]
print("\033[31mLANZANDO GRUPO 1\033[30m\n",flush=True,end="")
Launcher(th,proc)
sleep(5)

#Lanzamos 21 más y por tanto se levantará 2 veces la barrera
th =[]
funcs2 = [proc] * 21
print("\033[31mLANZANDO GRUPO 21\033[30m\n",flush=True,end="")
Launcher(th,*funcs2)
sleep(5)

# Y al final 9 más para que acaben todos

th =[]
funcs3 = [proc] * 9
print("\033[31mLANZANDO GRUPO 9\033[30m\n", flush=True,end="")
Launcher(th,*funcs3)

sleep(2)



### Interruptores (lightswitch)

Los interruptores sirven para proteger un recurso accesible para un conjunto de procesos pero no para el resto. Utiliza dos funciones lightswitch_on y lightswitch_off. El primer proceso del conjunto de procesos en entrar protege el recurso y el resto de procesos del mismo tipo pueden entrar y usarlo. En el lightswitch_off el último proceso en salir libera el recurso. 

In [ ]:
# Dado que el contador y el semáforo mutex son propios para cada interruptor pero comunes
# para ambas funciones en más fácil implementarlos como clases.

class Lightswitch:
    def __init__(self, resource):
        self.resource = resource
        self.counter = 0
        self.mutex = Semaphore(1)
        
    def on(self):
        down(self.mutex)
        self.counter += 1
        if self.counter == 1:
            down(self.resource)
        up(self.mutex)
        
    def off(self):
        down(self.mutex)
        self.counter -= 1
        if self.counter == 0:
            up(self.resource)
        up(self.mutex)
        
# Para probarlo vamos a poner un conjunto de procesos que van a usar el recurso y otros
# procesos que no están en el conjunto y que quieren usar también el recurso

# Semáforo que controla el recurso
resource = Semaphore(1)
lsw = Lightswitch(resource)

def resource_func(categoria):
    print("Estoy usando el recurso como "+categoria+"\n", flush= True,end="")
    sleep(randint(2,5))
    
# Usa el recurso directamente que es la funcion resource_func protegida por el semáforo
# resource

def suelto():
    print("\033[33mVoy a pedir el recurso como hilo suelto\033[00m\n", flush= True,end="")
    down(resource)
    print("\033[31mPido el recurso como hilo suelto\033[00m\n", flush= True,end="")
    resource_func("SUELTO")
    print("\033[35mLibero el recurso como hilo suelto\033[00m\n", flush= True,end="")
    up(resource)
    
def conjunto():
    global lsw
    print("\033[34mVoy a pedir el recurso como hilo de un grupo\033[00m\n", flush= True,end="")
    lsw.on()
    print("\033[32mPido el recurso como hilo de un grupo\033[00m\n", flush= True,end="")
    resource_func("CONJUNTO")
    print("\033[36mLibero el recurso como hilo de un grupo\033[00m\n", flush= True,end="")
    lsw.off()
    
# Entran primero uno suelto, después varios de tipo conjunto, después varios sueltos y finalmente 
# otro conjunto de procesos de tipo conjunto. Todos se lanzan en un breve espacio de tiempo 
# con lo cual esperan si es necesario.


th = []
Launcher(th, suelto)

th=[]
Launcher(th, conjunto, conjunto, conjunto, conjunto, conjunto)
th=[]
Launcher(th, suelto, suelto, suelto)
th=[]
Launcher(th, conjunto, conjunto, conjunto, conjunto)
    
    


## Problemas típicos con semáforos
### Problema del Productor - consumidor

#### Enunciado del productor-consumidor

Uno o varios hilos producen un bien que guardan en un almacén y un o varios hilos consumen ese bien del almacén. El almacé tendrá un tamaño limitado.

#### Implementación del productor-consumidor

Vamos a crear un productor y un consumidor con un almacén de tamaño limitado (N). Para que estos hilos no sean infinitos se sustituye el bucle infinito por un bucle de tamaño limitado. Así mismo, para poder controlar cuándo se ejecutan los dos hilos se añade un semáforo controlador para el productor y otro para el consumidor.

In [ ]:
N=24
INTENTOS = 30
NPRODCONS = 60
almacen = ['*'] * N
elemin = 0
elemout = 0

mutex = Semaphore(1)
llenos = Semaphore(0)
vacios = Semaphore(N)
prod = Semaphore(0)
cons = Semaphore(0)

def productor():
    global almacen, elemin, elemout
    i=0
    
    while i < NPRODCONS:
        i += 1
        down(prod) #auxiliar
        down(vacios)
        down(mutex)
        almacen[elemin] = choice(ascii_letters)
        elemin = (elemin+1)%N
        up(mutex)
        up(llenos)
    return
    
def consumidor():
    global almacen, elemin, elemout
    i=0
    
    while i < NPRODCONS:
        i += 1
        down(cons) #auxiliar
        down(llenos)
        down(mutex)
        a = almacen[elemout]
        almacen[elemout] = '*'
        elemout = (elemout+1)%N
        up(mutex)
        up(vacios)
    return

trs= []
Launcher(trs,productor, consumidor)

# Controlando la ejecución con los semáforos auxiliares
for i in range(26):
    up(prod)
    print(almacen)
    sleep(1)

for i in range(2):
    up(cons)
    print(almacen)
    sleep(1)
    

for i in range(5):
    up(cons)
    print(almacen)
    sleep(1)
    
sleep(3)
    
for i in range(INTENTOS):
    n_cons = randint(0,3)
    n_prod = randint(0,3)
    for _ in range(n_cons+n_prod):
        if randint(0,1):
            if n_cons == 0:
                up(prod)
            else:
                n_cons -= 1
                up(cons)
        else:
            if n_prod == 0:
                up(cons)
            else:
                n_prod -= 1
                up(prod)
    print(almacen)
    sleep(5)


#### Cuestiones

1. Intenta explicar el resultado.
2. ¿Qué sucede si se cambia el orden relativo del mutex respecto a los semáforos de vacíos y llenos. Pruébalo.
3. ¿El código es válido para múltiples productores y consumidores? Haz las pruebas necesarias.

## Problema del baile de salón
### Enunciado del problema del baile de salón
En el hotel Hastor de New York existe una sala de baile en la que los hombres se ponen en una fila y las mujeres en otra de tal forma que salen a bailar por parejas en el orden en el que están en la fila. Por supuesto ni un hombre ni una mujer pueden salir a bailar sólos ni quedarse en la pista sólos. Sin embargo no tienen por qué salir con la pareja con la que entraron.

In [ ]:
m1 = Semaphore(0)
m2 = Semaphore(0)
m3 = Semaphore(0)
m4 = Semaphore(0)

def Lider(m1,m2):
    up(m1)
    down(m2)
    
def Follower(m1,m2):
    down(m1)
    up(m2)
    
def Mujer(numero):
    Lider(m1,m2)
    Baila("mujer",numero)
    Follower(m3,m4)
    Sale("mujer",numero)

def Hombre(numero):
    Follower(m1,m2)
    Baila("hombre",numero)
    Lider(m3,m4)
    Sale("hombre",numero)
    
def Baila(genero, numero):
    print("Bailo, soy %s y mi número es %d\n" %(genero, numero),flush=True, end="")
    sleep(randint(1,3))
    
def Sale(genero, numero):
    print("Salgo, soy %s y mi número es %d\n" %(genero, numero),flush=True end="")
    sleep(randint(1,3))
    
def H0(): Hombre(0)
def H1(): Hombre(1)
def H2(): Hombre(2)
def H3(): Hombre(3)
def H4(): Hombre(4)
def H5(): Hombre(5)
def H6(): Hombre(6)
    
def M0(): Mujer(0)
def M1(): Mujer(1)
def M2(): Mujer(2)
def M3(): Mujer(3)
def M4(): Mujer(4)
def M5(): Mujer(5)
def M6(): Mujer(6)
    
trs= []
Launcher(trs,H0,H1,H2,H3,H4,H5,H6)
print("Hay hombres en la cola esperando\n",flush=True,end="")
sleep(5)
print("Aparecen tres mujeres en la cola\n",flush=True,end="")
trs= []
Launcher(trs,M0,M1,M2)
sleep(5)
print("Y ahora cuatro más mujeres en la cola\n",flush=True,end="")
trs= []
Launcher(trs,M3,M4,M5,M6)

#### Problema
Lanza distintos bailarines en el orden en el que desees y mira qué sucede. Cambia también el tiempo de baile en los distintos procesos. Por ejemplo, que a las mujeres les guste mucho más que a los hombres el quedarse bailando. ¿Qué sucede?

## Problema de los babuinos
### Enunciado del problema de los babuinos
En el parque nacional Kruger en Sudafrica hay un cañón muy profundo con una simple cuerda para cruzarlo. Los babuinos necesitan cruzar ese cañón constantemente en ambas direcciones gracias a una cuerda. Sin embargo:
- Como los babuinos son muy agresivos, si dos de ellos se encuentran en cualquier punto de la cuerda yendo en direcciones opuestas, estos se pelearán y terminarán cayendo por el cañón y muriendo.
- La cuerda no es muy resistente y aguanta a un máximo de cinco babuinos simultáneamente. Si en cualquier instante hay más de cinco babuinos en la cuerda, esta se romperá y los babuinos caerán también al vacío.

En las siguientes líneas vamos a crear los procedimientos para ser lanzados como hilos. Se puede comprobar que coinciden perfectamente con el pseudocódigo presenado en las clases de teoría.

In [ ]:
N = 5 # Número máximo de babuinos en la cuerda
maxBabuinosVa = Semaphore(N)
maxBabuinosViene = Semaphore(N)
cuerda = Semaphore(1)

def cruza(texto):
    print ("CRUZA %s\n" % texto, flush=True, end="")
    sleep(randint(1,3))

def BabuinoVa(clase):
    global maxBabuinosVa,cuerda
    down(maxBabuinosVa)
    clase.on("VA\n")
    cruza("VA\n")
    clase.off("VA\n")
    up(maxBabuinosVa)
    
def BabuinoViene(clase): 
    global maxBabuinosViene,cuerda
    down(maxBabuinosViene)
    clase.on("VIENE\n")
    cruza("VIENE\n") 
    clase.off("VIENE\n")
    up(maxBabuinosViene) 

Ya sólo nos falta probarlo. Para ello creamos hilos con babuinos que quieren ir o venir aleatoriamente. Creamos 20 de esos hilos.

In [ ]:
va = Lightswitch(cuerda)
viene = Lightswitch(cuerda)

for i in range(20):
    if randint(0,1):
        t=Thread(target=BabuinoVa,args=(va,))
    else:
        t=Thread(target=BabuinoViene,args=(viene,))
    t.start()

Como se puede ver, esta solución no previene inanición.

#### Cuestiones y problemas
1. Comprueba a partir de la salida que se ha resuelto correctamente el problema.
2. Intenta modificar el resultado para que no haya inanición.

## Problema de la cena de los filósofos
El problema de la cena de los filósofos (dining philosophers problem) es un problema clásico de las ciencias de la computación propuesto por Edsger Dijkstra en 1965 para representar el problema de la sincronización de procesos en un sistema operativo. Cabe aclarar que la interpretación está basada en pensadores chinos, quienes comían con dos palillos, donde es más lógico que se necesite el del comensal que se siente al lado para poder comer.

### Enunciado del problema

Cinco filósofos se sientan alrededor de una mesa y pasan su vida cenando y pensando. Cada filósofo tiene un plato de fideos y un tenedor a la izquierda de su plato. Para comer los fideos son necesarios dos tenedores y cada filósofo sólo puede tomar los que están a su izquierda y derecha. Si cualquier filósofo toma un tenedor y el otro está ocupado, se quedará esperando, con el tenedor en la mano, hasta que pueda tomar el otro tenedor, para luego empezar a comer.

Si dos filósofos adyacentes intentan tomar el mismo tenedor a una vez, se produce una condición de carrera: ambos compiten por tomar el mismo tenedor, y uno de ellos se queda sin comer.

Si todos los filósofos toman el tenedor que está a su derecha al mismo tiempo, entonces todos se quedarán esperando eternamente, porque alguien debe liberar el tenedor que les falta. Nadie lo hará porque todos se encuentran en la misma situación (esperando que alguno deje sus tenedores). Entonces los filósofos se morirán de hambre. Este bloqueo mutuo se denomina interbloqueo o _deadlock_.

El problema consiste en encontrar un algoritmo que permita que los filósofos no se mueran de hambre.

### Solución
A continuación se presenta una solución para el problema en el que no se previene inanición de algún filósofo.

In [ ]:
N = 6 # Número de filósofos
INTENTOS = 5

def izq(nfilosofo):
    return (nfilosofo -1) % N

def der(nfilosofo):
    return (nfilosofo +1) % N

estado = ["PENSANDO"]*N
semFilosofo = [0]*N
mutex = Semaphore(1)

def pensar(max,nfilosofo):
    print("PENSANDO %d\n" % nfilosofo,flush=True, end="")
    sleep(randint(0,max))
    
def comer(max,nfilosofo):
    print("COMIENDO %d\n" % nfilosofo,flush=True, end="")
    sleep(randint(0,max))
    
def intenta(nfilosofo):
    if estado[nfilosofo] == "HAMBRIENTO" and estado[izq(nfilosofo)] != "COMIENDO" and estado[der(nfilosofo)] != "COMIENDO":
        estado[nfilosofo] = "COMIENDO"
        up(semFilosofo[nfilosofo])
        
def cogeTenedores(nfilosofo):
    down(mutex)
    estado[nfilosofo]="HAMBRIENTO"
    intenta(nfilosofo)
    up(mutex)
    down(semFilosofo[nfilosofo])
    
def dejaTenedores(nfilosofo):
    down(mutex)
    estado[nfilosofo]="PENSANDO"
    intenta(izq(nfilosofo))
    intenta(der(nfilosofo))
    up(mutex)
    
def filosofo(nfilosofo):
    for j in range(INTENTOS):
        pensar(N,nfilosofo)
        cogeTenedores(nfilosofo)
        comer(N,nfilosofo)
        dejaTenedores(nfilosofo)
    
t = []
for i in range(N):
    semFilosofo[i] = Semaphore(0)
    t= Thread(target=filosofo,args=(i,))
    t.start()
    

#### Cuestiones
1. Identifica qué hacen los procedimientos __izq__ y __der__
2. El procedimiento más importante es el procedimiento __intenta__. Identifica su funcionamiento e identifica por qué cuando un filósofo deja los tenedores hace que lo intenten los filósofos que están a su izquierda y a su derecha.

## Otros problemas
En las siguientes celdas se van a enunciar un conjunto de problemas típicos para que pruebes a solucionarlos con las herramientas que ya dispones. Todos ellos han sido extraidos del libro ["El pequeño libro de los semáforos"](https://greenteapress.com/wp/semaphores/).

Para aprender bien se aconseja que añadas una o varias celdas después de cada problema y que sigas el siguiente procedimiento para resolverlos.
1. Analiza el problema en papel.
    1. Identifica los procesos que deben existir.
    2. Identifica los puntos de concurrencia y ordénalos en cada proceso.
    3. Encuentra una forma de solucionar cada punto de concurrencia.
    4. Añade los _mutex_ necesarios para proteger la memoria compartida para que no se produzcan condiciones de carrera.
    5. Analiza el código resultante y comprueba que no hay problemas de interbloqueos.
    6. Analiza el problema y comprueba si hay problemas de inanición y si estos son resolubles sin desaprovechar recursos.
2. Impleméntalo usando los recursos disponibles en este "_notebook_".
3. Añade las salidas necesarias para comprobar que funciona correctamente.
4. Si algo no funciona vuelve al punto 1.

### Problema de los lectores-escritores
Este problema se refiere a cualquier situación en la que una estructura de datos, base de datos o sistema de archivos es leído y modificado por hilos concurrentes. Mientras se está escribiendo o modificando la estructura de datos, a menudo es necesario impedir la lectura de otros hilos, para evitar que un lector interrumpa una modificación en curso y lea de forma inconsistente, o bien datos no válidos. Como en el problema productor-consumidor, la solución es asimétrica. Los lectores y escritores ejecutan códigos diferentes antes de entrar en la sección crítica. Las restricciones de sincronización son:
1. Cualquier número de lectores puede estar en la sección crítica simultáneamente.
2. Los escritores deben tener acceso exclusivo a la sección crítica. En otras palabras, un escritor no puede entrar en la sección crítica mientras que cualquier otro hilo (lector o escritor) está allí, y mientras el escritor está allí, ningún otro hilo puede entrar.

In [ ]:
cuenta_lectores = 0

mutex = Semaphore(1)
recurso = Semaphore(1)

def leyendo():
    print("\033[32mLeyendo\033[00m\n",flush=True,end="")
    sleep(randint(1,10))
    print("\033[32mDejando de leer\033[00m\n",flush=True,end="")
    
def escribiendo():
    print("\033[31mEscribiendo\033[00m\n",flush=True,end="")
    sleep(randint(1,10))
    print("\033[31mDejando de escribir\033[00m\n",flush=True,end="")
    
def lector():
    global cuenta_lectores
    
    down(mutex)
    cuenta_lectores += 1
    if cuenta_lectores == 1:
        down(recurso)
    up(mutex)
    
    leyendo()
    
    down(mutex)
    cuenta_lectores -= 1
    if cuenta_lectores == 0:
        up(recurso)
    up(mutex)
    
def escritor():
    down(recurso)
    escribiendo()
    up(recurso)
    
th1 = []
th2 = []
th3 = []
th4 = []
th5 = []

Launcher(th1,lector,lector,lector, escritor, lector)
sleep(9)
Launcher(th2,escritor,lector,lector, lector, lector)
sleep(9)
Launcher(th3,lector,lector,lector, escritor, lector)
sleep(9)
Launcher(th4,lector,lector,escritor, lector, lector)
sleep(9)
Launcher(th5,lector,lector,lector, lector, escritor)
    

### Problema de los lectores-escritores con prioridad a escritores
Es el mismo problema que el anterior pero si un escritor desea escribir no puede leer ningún lector nuevo hasta que no se realice la o las escrituras.

### Problema de la cena de los salvajes
Una tribu de salvajes realiza cenas comunales de una olla grande que puede contener M porciones de misioneros guisados. Cuando un salvaje quiere comer, se sirve de la olla, a menos que esté vacía. Si la olla está vacía, el salvaje despierta al cocinero que cocina a un nuevo misionero y luego espera hasta que el cocinero haya vuelto a llenar la olla. Evidentemente si otro quiere cenar, la olla está vacía pero el cocinero está despierto no tiene que despertarlo pero esperará a que llene la olla.

**Variante**: Se pueden concinar sólo hasta N misioneros que fueron el total de misioneros capturados en la última cacería de misioneros.

In [ ]:
sem_olla = Semaphore(1)
olla_vacia = Semaphore(0)
olla_llena = Semaphore(0)

TAMANO_MISIONERO = 20
porciones = 0

def llena_olla():
    global porciones
    print("¡Voy a llenar la olla!\n",flush=True,end="")
    porciones = TAMANO_MISIONERO
    
def como_misionero():
    sleep(randint(3,6))


def savage():
    global porciones
    while True:
        down(sem_olla)
        if porciones == 0:
            up(olla_vacia)
            down(olla_llena)
        porciones -= 1
        print("Quedan " +str(porciones)+ " porciones\n",flush=True,end="")
        up(sem_olla)
        como_misionero()
    
def cocinero():
    while True:
        down(olla_vacia)
        llena_olla()
        up(olla_llena)
        
th1 = []
th2 = []

Launcher(th1,cocinero)
print("Tengo llena la olla, ¡a comer!\n",flush=True,end="")
Launcher(th2,savage,savage,savage, savage, savage, savage)
print("Estamos comiendo 6 salvajes\n",flush=True,end="")
sleep(20)
    

### Problema del H<sub>2</sub>O
Este problema ha sido un elemento básico de la clase de sistemas operativos en la Universidad de California en Berkeley durante al menos una década. 

Hay dos tipos de átomos: oxígeno e hidrógeno, y queremos crear moléculas de agua. Para ello tenemos un catalizador que espera a que estén disponibles dos átomos de hidrógeno y uno de oxígeno para porducir una molécula de agua.
Para ensamblar estos átomos en moléculas de agua tenemos que crear una barrera que haga que cada átomo espere a que estén disponibles el resto  espere hasta que una molécula completa esté lista para proceder. A medida que cada hilo pasa la barrera, debe invocar la unión. Debes garantizar que todos los hilos de una molécula invoquen la unión antes de que lo haga cualquiera de los hilos de la siguiente molécula. Como los átomos son indistinguibles basta con que haya dos hidrógenos y un oxígeno para que puedan reaccionar.

In [ ]:
N=2

mutex = Semaphore(1)
oxigenos = Semaphore(0)
hidrogenos = Semaphore(0)
contador = 0

def junta():
    print("\n\033[36mAgua!!!!\033[00m\n",flush=True,end="")
    
def catalizador():
    for i in range(N):
        down(oxigenos)
        down(hidrogenos)
        junta()
        
def oxigeno():
    print("Oxigeno\n",flush=True,end="")
    up(oxigenos)
    
def hidrogeno():
    global contador
    print("Hidrogeno\n",flush=True,end="")
    down(mutex)
    contador += 1
    if contador %2 == 0:
        up(hidrogenos)
        contador -= 2
    up(mutex)
    

t1 = []
t2 =[]
t3 = []
t4 =[]

Launcher(t1,catalizador)
Launcher(t2,oxigeno,oxigeno)
sleep(5)
Launcher(t3,hidrogeno, hidrogeno,hidrogeno)
sleep(5)
Launcher(t4,hidrogeno)

### Problema de la montaña rusa
Supongamos que hay n de pasajeros y un coche. Los pasajeros esperan repetidamente para dar una vuelta en la montaña rusa, que puede contener C pasajeros. El coche de la montaña rusa espera a estar lleno para empezar el viaje.

Aquí hay algunos detalles adicionales:
- Los pasajeros deben invocar el _embarque_ y _desembarque_.
- El coche debe invocar la _carga_, _corre_ y _descarga_.
- Los pasajeros no pueden embarcar hasta que el coche haya invocado la carga.
- El coche no puede salir hasta que se haya llenado.
- Los pasajeros no pueden desembarcar hasta que el coche no haya invocado la descarga.

In [ ]:
contador_pasajeros_S = 0
contador_pasajeros_B = 0
N = 5
vueltas = 7

mutexS = Semaphore(1)
mutexB = Semaphore(1)
cargar = Semaphore(0)
cargados = Semaphore(0)
descargar = Semaphore(0)
descargados = Semaphore(0)

def embarque():
    global contador_pasajeros_S
    down(mutexS)
    contador_pasajeros_S += 1
    if contador_pasajeros_S == 1:
        down(cargar)
        print("Empezando a embarcar\n",flush=True,end="")
    if contador_pasajeros_S == N:
        contador_pasajeros_S = 0
        print("Embarcando el último\n",flush=True,end="")
        up(cargados)
    up(mutexS)

def carga():
    up(cargar)
    print("Cargando\n",flush=True,end="")
    down(cargados)
    
def desembarque():
    global contador_pasajeros_B
    down(mutexB)
    contador_pasajeros_B += 1
    if contador_pasajeros_B == 1:
        down(descargar)
        print("Empezando a desembarcar\n",flush=True, end="")
    if contador_pasajeros_B == N:
        contador_pasajeros_B = 0
        print("Desembarcando el último\n",flush=True,end="")
        up(descargados)
    up(mutexB)
    
def descarga():
    up(descargar)
    print("Descargando\n",flush=True,end="")
    down(descargados)

def corre():
    print("\033[31mCorriendo\033[00m\n",flush=True,end="")
    sleep(5)
    
def pasajero():
    embarque()
    desembarque()
    
def coche():
    print("\033[32mEmpiezan los viajes\033[00m\n",flush=True,end="")
    for i in range(vueltas):
        carga()
        corre()
        descarga()
    print("\033[32mSe acabaron los viajes\033[00m\n",flush=True,end="")
    
    
t1 = []
t2 =[]
t3 = []
t4 =[]

Launcher(t1,coche)
Launcher(t2,pasajero,pasajero,pasajero,pasajero)
sleep(5)
Launcher(t3,pasajero,pasajero,pasajero)
sleep(5)
Launcher(t4,pasajero,pasajero,pasajero)

pasajeros = [pasajero]*25

t5 =[]
Launcher(t5,*pasajeros)

### Problema del bar de sushi
Imagina un bar de sushi con 5 asientos. Si llega un cliente y hay un asiento vacío, puede tomar asiento inmediatamente. Pero si cuando llega cuando los 5 asientos están llenos, eso significa que pueden están cenando juntos, y tendrá que esperar a que todos se vayan antes de sentarse.

In [ ]:
asientos_ocupados = 0
N = 5

mutex = Semaphore(1)
mutex2 = Semaphore(1)
sentarse = Semaphore(1)

def me_siento():
    print("\033[31mSentandome\033[00m\n",flush=True,end="")
    sleep(randint(1,10))
    print("\033[32mLevantándome\033[00m\n",flush=True,end="")
    
def cliente():
    global N, asientos_ocupados
    
    down(mutex2)
    down(sentarse)
    up(sentarse)
    down(mutex)
    asientos_ocupados += 1
    if asientos_ocupados == N:
        down(sentarse)
    up(mutex)
    up(mutex2)
    
    me_siento()
    
    down(mutex)
    asientos_ocupados -= 1
    if asientos_ocupados == 0:
        up(sentarse)
    up(mutex)

th1 = []
Launcher(th1,cliente,cliente,cliente)
sleep(5)

th2 = []
Launcher(th2,cliente,cliente,cliente,cliente,cliente,cliente)


### La tarta de cumpleaños

En un cumpleaños de ninos pequeños se pretende repartir trozos de tarta a todos los niños. Como un día
especial que es, los niños pueden repetir tantas veces como quieran y cuando quieran pero usando un sistema de rondas, de tal forma que hasta que todos los niños no han recibido su trozo de tarta de una ronda, los niños que ya han tomado tarta no pueden repetir.

Implementa el proceso **ninoPideTarta(int NumNino)** teniendo en cuenta que los semáforos que se van a utilizar son semáforos blandos (tambien denominados debiles). Se dispone de la funcion auxiliar **comeTarta()** para representar la tarea de comer el trozo de tarta. De igual forma, se utilizarán las funciones **up(semaforo)** y **down(semaforo)** para manipular los semáforos. Se pueden crear otras funciones si se consideran necesarias.

In [ ]:
from random import shuffle

N = 10
maximo = 5

n0 = Semaphore(1)
n1 = Semaphore(1)
n2 = Semaphore(1)
n3 = Semaphore(1)
n4 = Semaphore(1)
n5 = Semaphore(1)
n6 = Semaphore(1)
n7 = Semaphore(1)
n8 = Semaphore(1)
n9 = Semaphore(1)
ninnos = SemaphoreArray(N,value=1)
mutex = Semaphore(1)

cuenta = 0

def tarta(i):
    print("\033[32mEl niño %d ha pedido un trozo de tarta\033[00m" % i,flush=True)

def comeTarta(i):
    print("\033[36mEl niño %d está comiéndose la tarta\033[00m" % i,flush=True)
    sleep(randint(1,10))
    print("\033[36mEl niño %d se ha acabado la tarta\033[00m" % i,flush=True)

def ninoPideTarta(i):
    global N, cuenta, ninnos
    orden = [0,1,2,3,4,5,6,7,8,9] # Para simular el semáforo blando se despiertan según este orden
    for c in range(maximo): # Les dejamos comer un número máximo de veces
        down(ninnos[i]) # La vez siguiente se dormirá si no han comido tarta todos los niños
                        # Se le da un trozo de tarta
        tarta(i)
        down(mutex)
        cuenta += 1 # Cuenta los niños
        if cuenta == N: # Si todos los niños han pedido tarta ...
            cuenta = 0
            shuffle(ninnos) # Se cambia aleatoriamente el orden para simular blandos
            for i in range(N): # Se les deja volver a pedir a todos
                up(ninnos[i]) 
        up(mutex)
        comeTarta(i)
        
def P0(): ninoPideTarta(0)
def P1(): ninoPideTarta(1)
def P2(): ninoPideTarta(2)
def P3(): ninoPideTarta(3)
def P4(): ninoPideTarta(4)
def P5(): ninoPideTarta(5)
def P6(): ninoPideTarta(6)
def P7(): ninoPideTarta(7)
def P8(): ninoPideTarta(8)
def P9(): ninoPideTarta(9)

th = []

Launcher(th,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9)

### Problema de Santa Klaus

Santa Klaus es un hombre muy mayor y necesita dormir mucho; por ello aprovecha siempre que puede para
dormir.

Los elfos saben de su necesidad y han decidido que aunque haya un problema en la fábrica de juguetes no
le despertarán, pero cuando haya tres problemas entonces sí que lo haran.

La fábrica y la oficina de Santa estan muy alejadas, por lo que si un elfo va en busca de Santa no vuelve hasta que Santa resuelva su problema.

Los elfos irán a despertar a Santa pero si no hay otros dos elfos esperando es que no hay suficientes problemas como para despertar a Santa, por ello y ya que no tienen nada que hacer se quedarán a la espera. El tercer elfo hará lo mismo pero despertara a Santa. Cuando Santa resuelva los problemas en la fábrica los tres elfos regresarán a la fábrica.

Así mismo los renos estan de vacaciones todo el año y vuelven por navidad para tirar del trineo de Santa. Cuando llegue el ultimo de los 9 renos despertará a Santa para irse a repartir los juguetes.

No es necesario controlar lo que sucede con los elfos y la produccion cuando Santa esta de reparto.

In [ ]:
NE = 3
NR = 9

mutex = Semaphore(1)
mutexElfo = Semaphore(1)
mutexReno = Semaphore(1)
elfo = Semaphore(0)
reno = Semaphore(0)
santa = Semaphore(0)
estado =[]
nrenos = 0
nelfos = 0

def resuelveProblemas():
    print("\033[31mResolviendo problemas\033[00m\n",flush=True,end="")
    sleep(randint(1,5))
    print("\033[31mResueltos\033[00m\n",flush=True,end="")
    
def reparteRegalos():
    print("\033[32mRepartiendo regalos\033[00m\n",flush=True,end="")
    sleep(randint(30,60))
    print("\033[32mRepartidos\033[00m\n",flush=True,end="")

def produce():
    sleep(randint(1,50))

def vacaciones():
    sleep(randint(30,40))
    
    
def elfoTrabaja():
    global estado,NE,nelfos
    for i in range(5):
        produce()
        down(mutexElfo)
        print("Problema elfo\n",flush=True,end="")
        nelfos += 1
        if nelfos >= NE:
            nelfos -= NE
            down(mutex)
            estado.append("ELFOS")
            up(mutex)
            up(santa)
        up(mutexElfo)
        down(elfo)
    
def renoViene():
    global estado,NR,nrenos
    vacaciones()
    down(mutexReno)
    print("Llega reno\n",flush=True,end="")
    nrenos += 1
    if nrenos == NR:
        nrenos = 0
        down(mutex)
        estado.append("RENOS")
        up(mutex)
        up(santa)
    up(mutexReno)
    down(reno)
    
def santaKlaus():
    global estado, NR, NE
    for a in range(36):
        down(santa)
        down(mutex)
        if "RENOS" in estado:
            ind = estado.index("RENOS")
            estado = estado[0:ind]+estado[ind+1:]
            up(mutex)
            for i in range(NR): up(reno)
            reparteRegalos()
        elif estado[0] == "ELFOS":
            estado = estado[1:]
            up(mutex)
            for i in range(NE): up(elfo)
            resuelveProblemas()
        
threads = []
for i in range(21):
    t = Thread(target=elfoTrabaja)
    threads.append(t)
for i in range(9):
    t = Thread(target=renoViene)
    threads.append(t)
t = Thread(target=santaKlaus)
threads.append(t)
for thr in threads:
    thr.start()

### El baño unisex de la EHS

En la Escuela de Hacking Superior todos los aseos son unisex. Para usar estos aseos hay un selector a la entrada que tiene tres posiciones: hombres, mujeres y libre. El cartel de hombres lo pone el primer hombre que entre, el de mujeres la primera mujer que entre y el de libre lo pone cualquiera al salir siempre que quede vacío el aseo. Cuando esta puesto el cartel de mujeres sólo pueden entrar mujeres y cuando pone el cartel de hombres sólo pueden entrar hombres. En el aseo sólo caben 10 personas por lo que cuando está lleno se activara un cartel que indica que está lleno y los que lleguen esperarán fuera a que haya sitio.

Resuelve  el  problema  implementando  el  pseudocodigo  de  los  procesos **mujer()** y de **hombre()** para que cumplan las condiciones. No es necesario prevenir la inanición.

In [ ]:
N=10
maximoH = Semaphore(N)
maximoM = Semaphore(N)
mutexM = Semaphore(1)
mutexH = Semaphore(1)

aseo = Semaphore(1)

hombresC = 0
mujeresC = 0

def usandoAseoM():
    print("Usando (\033[31mM\033[00m)\n",flush=True,end="")
    sleep(randint(1,3))
    print("Saliendo (\033[31mM\033[00m)\n",flush=True,end="")
    
def usandoAseoH():
    print("Usando (\033[34mH\033[00m)\n",flush=True,end="")
    sleep(randint(1,3))
    print("Saliendo (\033[34mH\033[00m)\n",flush=True,end="")
    
def M():
    global mujeresC
    sleep(randint(1,40))
    down(maximoM)
    down(mutexM)
    mujeresC += 1
    if mujeresC == 1:
        down(aseo)
        print("\033[31mMujeres\033[00m\n",flush=True,end="")
    up(mutexM)
    
    usandoAseoM()
    
    down(mutexM)
    mujeresC -= 1
    if mujeresC == 0:
        print("\033[32mLibre\033[00m\n",flush=True,end="")
        up(aseo)
    up(mutexM)
    up(maximoM)
    
def H():
    global hombresC
    sleep(randint(1,40))
    
    down(maximoH)
    down(mutexH)
    hombresC += 1
    if hombresC == 1:
        down(aseo)
        print("\033[34mHombres\033[00m\n",flush=True,end="")
    up(mutexH)
    
    usandoAseoH()
    
    down(mutexH)
    hombresC -= 1
    if hombresC == 0:
        print("\033[32mLibre\033[00m\n",flush=True,end="")
        up(aseo)
    up(mutexH)
    up(maximoH)
    
th1 = []
th2 = []

Launcher(th1,H,H,H,H,H,H,H,H,H,H,H,H,H)
Launcher(th2,M,M,M,M,M,H,M,M,M,M,M,M,M,M,M,M,M,M,M,M)

### Cruce automático

En el ano 2056 se realiza una modificación en el código de la circulación en la que cambia la regulación de los cruces con bajo tráfico. Dada la capacidad de las comunicaciones inalambricas cuando un coche se acerca a un cruce  reserva los elementos necesarios del cruce para poder realizar la maniobra.

Si no puede acceder a una de las zonas necesarias para realizar la maniobra no accederá al cruce. Es muy importante que si las maniobras de varios coches no interfieren todos los coches puedan acceder al cruce sin ningún problema.

Crea el proceso **coche(origen,destino)** donde origen y destino determinan las zonas a utilizar. En el esquema adjunto se puede ver la numeración de zonas así como la numeración de origen/destino. Por ejemplo, si el origen es 2 y el destino es 1 es necesario reservar las zonas Z3, Z4, Z1, sin embargo de 1 a 2 hace falta reservar solo Z2. 

Utiliza las primitivas **up(sem)**, **down(sem)**, **up(sem,n)** y **down(sem,n)** (las dos últimas para hacer ups y downs multiples si es necesario). No es necesario prevenir inanición pero sí evitar interbloqueos.

In [ ]:
Z = SemaphoreArray(4,1)
mutex = Semaphore(1)

def cruza():
    sleep(randint(1,2))
    

def coche(id,origen,destino):
    if destino < origen:
        destino += 4
    down(mutex)
    for i in range(origen,destino): 
        down(Z[i%4])
        print("\033[32mOcupada Z%d\033[00m\n" % (i%4+1),flush=True, end="")
    up(mutex)
    
    dest = destino
    if dest > 4: dest = destino - 4
    print("Coche %d cruza de %d a %d\n" % (id,origen,dest),flush=True, end="")
    cruza()
    
    print("Crucé %d" % id)
    for i in range(origen,destino):
        up(Z[i%4])
        print("\033[32mLibre Z%d\033[00m\n" % (i%4+1),flush=True, end="")

threads = []
for i in range(20):
    orig = randint(1,4)
    dest = randint(1,4)
    while dest == orig: dest = randint(1,4)
    t = Thread(target=coche,args=(i,orig,dest))
    threads.append(t)
for thr in threads:
    thr.start()

### Cadena de montaje
En una fábrica de coches hay 5 cadenas de montaje. De cada una de ellas salen los coches de uno en uno. Estos coches son almacenados en una zona de espera en la que caben 10 coches. En esta zona de espera hay 3 dársenas donde paran los camiones de transporte que pueden transportar cada uno 6 coches. Los camiones nunca parten antes de estar completamente cargados. Cuando un camion desea cargar puede hacerlo siempre y cuando uno que estuviese cargando pueda terminar la carga.

Implementa los procesos **cadenaMontaje** y **cargaCamion**.

**No es necesario** controlar las posiciones donde se encuentran los coches a cargar ni implementar el acceso a las darsenas ni prevenir inanición.

In [ ]:
C = 5
D = 3
N = 10
P = 20
S = 6

mutex = Semaphore(1)
mutex2 = Semaphore(1)
mutex3 = Semaphore(1)
colas = Semaphore(1)
llenos = Semaphore(0)
vacios = Semaphore(N)
darsenas = Semaphore(D)

llenosC = SemaphoreArray(3,0)
vaciosC = SemaphoreArray(3,S)

almacen=['-']*N
cola = [['-']*S]*D
darsenasC = ["LIBRE"]*D
introduccion = 0
salida = 0

def cadenaMontaje(i):
    global introduccion,almacen
    for i in range(12):
        sleep(randint(2,4))
        down(vacios)
        down(mutex)
        almacen[introduccion] = 'c'
        introduccion = (introduccion + 1) % N
        print("Almacén: %s\n" % almacen,flush=True, end="")
        up(mutex)
        up(llenos)
    
def marcarCoches(i):
    global salida,cola,almacen
    down(colas)
    
    for pos in range(S):
        down(llenos)
        down(mutex)
        valor = almacen[salida]
        almacen[salida] ='-'
        salida = (salida + 1) % N
        up(mutex)
        up(vacios)

        down(vaciosC[i])
        down(mutex2)
        cola[i][pos] = valor
        up(mutex2)
        up(llenosC[i])
    
    up(colas)
    print("Almacén: %s\n" % almacen,flush=True, end="")
    
def embarcando(dars):
    print("\033[31mEmbarcando dársena %d\033[00m\n" % dars,flush=True, end="")
    sleep(randint(1,3))
    print("\033[32mEmbarcado dársena %d\033[00m\n" % dars,flush=True, end="")
    
def cargaCamion():
    down(darsenas)
    
    down(mutex3)
    darsn = darsenasC.index("LIBRE")
    darsenasC[darsn] = "OCUPADO"
    print("Camión en dársena %d\n" % darsn,flush=True, end="")
    up(mutex3)
    marcarCoches(darsn)
    
    for pos in range(S):
        down(llenosC[darsn])
        down(mutex2)
        valor = cola[darsn][pos]
        up(mutex2)
        up(vaciosC[darsn])
        embarcando(darsn)
    
    darsenasC[darsn] = "LIBRE"
    up(darsenas)
    
threads = []
for i in range(C):
    t = Thread(target=cadenaMontaje,args=(i,))
    threads.append(t)
for thr in threads:
    thr.start()
    
sleep(3)
camiones = [cargaCamion]*10
th = []
Launcher(th,*camiones)

### Visita a la Sagrada Familia y museo

La basílica de la Sagrada Familia en Barcelona dispone de un museo en el que se explican los métodos de construcción y maquetas de prueba de carga. los responsables de la visita de turistas a la basílica están hartos de que los turistas entren a ver la basílica sin saber lo que están viendo y han decidido que es necesario visitar el museo antes de entrar en la basílica. Además, han decidido que la basílica solo se pueda visitar con un guía que explique lo que se está viendo. Nos piden implementar un sistema que permita realizar esto de forma eficiente y con las siguientes condiciones:
* El número maximo de personas en el museo es de 100 personas.
* Cuando un visitante abandona el museo se pone en la cola de espera para que los recoja un guía.
* Hay 5 guías.
* Un guía recoge a grupos de 10 personas.
* Si no hay 10 personas en la cola de espera el guía espera a que las haya y si hay más solo coge a 10

In [ ]:
NM = 100
NG = 10
G = 5
cuenta = 0

visita = Semaphore(NM)
mutex = Semaphore(1)
guia = Semaphore(0)
grupo = Semaphore(0)
basilica = Semaphore(0)

def llega():
    sleep(randint(1,100))
    print("Llega visitante\n",flush=True,end="")
    
def visita_museo():
    print("\033[34mVisitando museo\033[00m\n",flush=True,end="")
    sleep(randint(5,8))
    print("\033[44mVisitado museo\033[00m\n",flush=True,end="")
    
def visita_basilica():
    print("\033[32mVisitando basilica\033[00m\n",flush=True,end="")
    sleep(randint(20,30))
    print("\033[42mVisitada basilica\033[00m\n",flush=True,end="")
    up(basilica)

def guia_basilica():
    print("\033[31mInicia guía\033[00m\n",flush=True,end="")
    
def termina_guia():
    global NG
    for i in range(NG): down(basilica)
    print("\033[41mFinaliza guía\033[00m\n",flush=True,end="")

def Visitante():
    global NG, cuenta
    llega()
    down(visita)
    visita_museo()
    up(visita)
    
    down(mutex)
    cuenta += 1
    if cuenta >= NG:
        cuenta -= NG
        up(guia)
    up(mutex)
    
    down(grupo)
    visita_basilica()
    
    
def Guia():
    global NG
    
    for g in range(5):
        down(guia)
        guia_basilica()
        for i in range(NG): up(grupo)
        termina_guia()
    
procesos = [Visitante]*200 + [Guia]*G
th=[]
Launcher(th, *procesos)

### El aeropuerto de Umea

Umea es una pequeña ciudad sueca en el círculo polar ártico. Debido a las inclemencias climáticas su principal medio de transporte es un pequeño avión con un máximo de 32 plazas para viajeros. El pequeño aeropuerto tiene una única terminal de embarque. Con el fin de obtener rentabilidad en la línea de comunicación con Estocolmo y reducir costes, la línea aérea se usa de forma muy similar a la de una línea de autobuses pero por seguridad con unas pocas reglas que es necesario cumplir:
1.  Cuando el avión está preparado, éste se llena con los pasajeros que estén esperando. Si hay más de 32 pasajeros esperando se llenará con 32 sin un orden determinado. Los pasajeros que no hayan podido embarcar esperarán al siguiente avión.
2.  Si no hubiera pasajeros esperando, el avión volverá de forma inmediata a Estocolmo.
3.  Si los pasajeros están embarcando nadie que llegue después de que se inicie el embarque podrá entrar en la zona de embarque y mucho menos embarcar (independientemente de si los pasajeros llenan el avión o no) y sólo podrán entrar en la zona de embarque tras la partida del avión.

In [ ]:
embarcando = Semaphore(1)
embarcado = Semaphore(0)
embarque = Semaphore(0)
mutex = Semaphore(1)

cuenta = 0

def avion_sale(total):
    print("\033[32mAvión saliendo con %d pasajeros\033[00m\n" % total,flush=True,end="")

def pasajero():
    global cuenta
    sleep(randint(1,20))
    down(embarcando)
    up(embarcando)
    down(mutex)
    cuenta += 1
    print("Ha llegado un pasajero y hay %d\n" % cuenta,flush=True,end="")
    up(mutex)
    down(embarque)
    print("Embarcando\n",flush=True,end="")
    up(embarcado)
    
def avion():
    global cuenta
    down(embarcando)
    down(mutex)
    if cuenta < 32: 
        ups = cuenta
        cuenta = 0
    else: 
        ups = 32
        cuenta -= 32
    up(mutex)
    for i in range(ups):
        up(embarque)
    for i in range(ups):
        down(embarcado)
    up(embarcando)
    avion_sale(ups)
    
pasajeros = [pasajero]*200

th = []
Launcher(th,*pasajeros)
    
for a in range(15):
    sleep(randint(1,3))
    avion()

### Los pasos de cebra de Navalaoveja

En el ayuntamiento de Navalaoveja se han gastado todo el presupuesto en mármol. Como no tienen presupuesto suficiente han despedido a policías municipales pero aún así tienen que mantener la seguridad en los pasos de cebra a la salida de los colegios. Como no hay presupuesto suficiente nos piden que implementemos un sistema basado en semáforos para que tanto los peatones como los automóviles cumplan con las condiciones siguientes sin necesidad de policías:


* Si no hay automóvil que quieran acceder al paso de cebra los peatones deben poder cruzar cuando lo deseen.
* Si no hay peatones que quieran acceder al paso de cebra los automóviles deben poder pasar cuando lo deseen.
* Si hay diez automóviles esperando a que pasen peatones deben dejar de cruzar peatones y pasar los 10 coches. Aunque haya más coches, sólo pasarán 10 si hay peatones esperando.
* Salvo en el caso anterior los peatones siempre tienen prioridad para cruzar.
* Como el uso del paso de cebra en ambas direcciones tanto de peatones como de automóviles no es exclusivo, no se tendrá en cuenta la dirección en la que desean pasar.
* Evidentemente los peatones pueden pasar en grupos pero los coches sólo pueden pasar de uno en uno. 

    Implementa los procesos **peaton** y **coche()** usando semáforos. Para ello puedes usar las funciones up(sem), down(sem). También puedes usar las funciones llegapeaton(), llegacoche(), cruzapeaton(), cruzacoche(), continuapeaton() y continuacoche().

In [ ]:
cuenta_lectores = 0

mutex = Semaphore(1)
recurso = Semaphore(1)

def cruzapeaton():
    print("\033[32mCruzando peatón\033[00m\n",flush=True,end="")
    sleep(randint(1,3))
    print("\033[32mPeatón ha cruzado\033[00m\n",flush=True,end="")
    
def cruzacoche():
    print("\033[31mCruzando coche\033[00m\n",flush=True,end="")
    sleep(randint(1,3))
    print("\033[31mCoche ha cruzado\033[00m\n",flush=True,end="")
    
def peaton():
    global cuenta_lectores
    
    down(mutex)
    print("Llega peatón\n",flush=True,end="")
    cuenta_lectores += 1
    if cuenta_lectores == 1:
        down(recurso)
    up(mutex)
    
    cruzapeaton()
    
    down(mutex)
    cuenta_lectores -= 1
    print("Sale peatón\n",flush=True,end="")
    if cuenta_lectores == 0:
        up(recurso)
    up(mutex)
    
def llega_coche():
    print("Llega coche\n",flush=True,end="")
    
def sale_coche():
    print("Sale coche\n",flush=True,end="")
    
def coche():
    llega_coche()
    down(recurso)
    cruzacoche()
    up(recurso)
    sale_coche()
    
    
c1 = [coche,coche,peaton,coche,coche,peaton,coche,coche,coche,peaton,coche,coche]
th=[]
Launcher(th,*c1)

### Escuela de Parrandeo y Saraos
En la Escuela de Parrandeo y Saraos se organiza un examen de parrandeos y otro de saraos. Las condiciones del examen son las siguientes:

* Hay estudiantes de tres tipos: estudiantes que tienen los dos exámenes (28), estudiantes que tienen el examen de parrandeo (34) y estudiantes que tienen el examen de saraos(18).
* Todos los exámenes son orales y los hacen dos profesores en sendas aulas (cada uno en la suya). En cada aula solo hay 10 sillas para estudiantes, incluyendo la de quien está siendo examinado. Dentro de cada aula los estudiantes sólo pueden esperar sentados. El resto de los estudiantes esperarán fuera de las aulas.
* Los estudiantes que tienen los dos exámenes podrán entrar en cualquiera de los exámenes primero y una vez examinados podrán entrar en el otro (el orden es indiferente, no se basará en ninguna condición).
* Ambos profesores saben a cuántos estudiantes tienen que examinar y por tanto esperarán hasta haber evaluado a todos. Solo cuando terminen con todos se irán.

Crea las funciones principales de cada uno de los procesos: _profSaraos()_, _profParrandeo()_, _estSaraos()_, _estParrandeo()_ y _estSaraoParrandeo()_.

In [ ]:
ESTUDIANTES_SARAOS = 18
ESTUDIANTES_PARRANDEOS = 34
ESTUDIANTES_AMBOS = 28
NSILLAS_SARAOS = 10
NSILLAS_PARRANDEOS = 10

exSaraos = Semaphore(0)
evaluadoSaraos = Semaphore(0)
sillasSaraos = Semaphore(NSILLAS_SARAOS)

exParrandeos = Semaphore(0)
evaluadoParrandeos = Semaphore(0)
sillasParrandeos = Semaphore(NSILLAS_PARRANDEOS)

meestoyexaminando = [Semaphore(1)]*ESTUDIANTES_AMBOS

estudS = 0
estudP = 0
m1 = Semaphore(1)
m2 = Semaphore(1)

def llegaExamen(tipo):
    sleep(randint(1,3))
    print("Llegó %s\n" % tipo,flush=True,end="")
    
def presentaExamen(tipo, color1, color2):
    print("\033[" + color1 + "mExaminando de " + tipo + "\033[00m\n",flush=True,end="")
    sleep(randint(2,9))
    print("\033[" + color2 + "mExaminado de " + tipo + "\033[00m\n",flush=True,end="")
    
def estudianteuno(tipo,sillas,examen,evaluado, color1, color2):
    llegaExamen(tipo)
    down(sillas)
    print("Sentado %s\n" % tipo,flush=True,end="")
    down(examen)
    up(sillas)
    presentaExamen(tipo, color1, color2)
    up(evaluado)

def estudiantedosSaraos():
    global estudS
    
    down(m1)
    i = estudS
    estudS += 1
    up(m1)
    
    llegaExamen("Saraos")
    down(sillasSaraos)
    down(meestoyexaminando[i])
    print("Sentado Saraos\n",flush=True,end="")
    down(exSaraos)
    up(sillasSaraos)
    presentaExamen("Doble Sarao","33","32")
    up(evaluadoParrandeos)
    up(meestoyexaminando[i])
    
def estudiantedosParrandeos():
    global estudP
    
    down(m2)
    i = estudP
    estudP += 1
    up(m2)
    
    llegaExamen("Parrandeos")
    down(sillasParrandeos)
    down(meestoyexaminando[i])
    print("Sentado Parrandeos\n",flush=True,end="")
    down(exParrandeos)
    up(sillasParrandeos)
    presentaExamen("Doble Parrandeos","33","32")
    up(evaluadoParrandeos)
    up(meestoyexaminando[i])
    
    
# def estudiantesdos() Dos procesos y un estado Cuando se sienta en uno en el otro si 
# puede continuar se hace deja la silla, se pone a dormir y espera

def profesor(n_estudiantes,examen,evaluado):
    for i in range(n_estudiantes):
        up(examen)
        down(evaluado)

def profSaraos():
    profesor(ESTUDIANTES_AMBOS+ESTUDIANTES_SARAOS,exSaraos,evaluadoSaraos)
    
def profParrandeos():
    profesor(ESTUDIANTES_AMBOS+ESTUDIANTES_PARRANDEOS,exParrandeos,evaluadoParrandeos)

def estSaraos():
    estudianteuno("Saraos",sillasSaraos,exSaraos,evaluadoSaraos,"34","36")
    
def estParrandeos():
    estudianteuno("Parrandeos",sillasParrandeos,exParrandeos,evaluadoParrandeos,"31","35")
    
c1 = [profSaraos, profParrandeos]
for i in range(ESTUDIANTES_SARAOS):
    c1.append(estSaraos)
for i in range(ESTUDIANTES_PARRANDEOS):
    c1.append(estParrandeos)
for i in range(ESTUDIANTES_AMBOS):
    c1.append(estudiantedosParrandeos)
    c1.append(estudiantedosSaraos)
th=[]
Launcher(th,*c1)

### Problema del cuidado de niños

En una guardería comunitaria tiene que haber un adulto por cada tres niños. Los adultos esperan a tener un grupo de tres niños para entrar en la guardería y encargarse de ellos. No es necesario prevenir inanición.

In [ ]:
padre_s = Semaphore(0)
mutex = Semaphore(1)

def cuida():
    sleep(20)
    
def soy_cuidado():
    sleep(20)
    
def padre():
    down(mutex)
    down(padre_s)
    down(padre_s)
    down(padre_s)
    up(mutex)
    cuida()
    
def nino():
    up(padre_s)
    soy_cuidado()
    

### Problema ampliado del cuidado de niños

En una guardería comunitaria tiene que haber un adulto por cada tres niños. Los adultos esperan a tener un grupo de tres niños para entrar en la guardería y encargarse de ellos. Cuando tres niños se marchan un adulto puede marcharse (se supone que los niños que tuviese a cargo pasan a aquellos que no tienen tres niños en su grupo. No es necesario prevenir inanición.

### Problema generalizado del cuidado de niños

En una guardería comunitaria tiene que haber un adulto por cada tres niños. Los adultos se encargan del primer niño y cuando tienen 3 se hace necesario un nuevo adulto para cuidar de los siguientes. Cuando tres niños se marchan un adulto puede marcharse (se supone que los niños que tuviese a cargo pasan a aquellos que no tienen tres niños en su grupo. No es necesario prevenir inanición.

### Problema de los buscadores, insertadores y borradores

Hay tres tipos de hilos que comparten el acceso a una lista enlazada: los buscadores, los insertadores y los borradores. Los buscadores se limitan a examinar la lista, por lo que pueden ejecutarse simultáneamente. Los insertadores añaden nuevos elementos al final de la lista; las inserciones deben ser mutuamente excluyentes para evitar que dos insertadores inserten nuevos elementos al mismo tiempo. 

Sin embargo, un insertador puede proceder en paralelo con cualquier número de búsquedas. Por último, los borradores eliminan elementos de cualquier parte de la lista. Como máximo, un proceso de borrado puede acceder a la lista a la vez, y el borrado también debe ser mutuamente excluyente con las búsquedas y las inserciones.

Para ello se aportan las clases necesarias que es necesario modificar para que funcionen correctamente.

In [ ]:
# Creamos la clase node
class node:
    def __init__(self, data = None, next = None):
        self.data = data
        self.next = next

# Creamos la clase linked_list
class linked_list: 
    def __init__(self):
        self.head = None

    # Método para verificar si la estructura de datos esta vacia
    def is_empty(self):
        return self.head == None

    # Método para agregar elementos al final de la linked list
    def add_at_end(self, data):
        if not self.head:
            self.head = node(data=data)
            return
        curr = self.head
        while curr.next:
            curr = curr.next
        curr.next = node(data=data)
    
    # Método para buscar nodos
    def search_node(self, key):
        curr = self.head
        prev = None
        while curr and curr.data != key:
            prev = curr
            curr = curr.next
        return prev,curr
    
    # Método para eliminar nodos
    def delete_node(self, key):
        curr = self.head
        prev = None
        while curr and curr.data != key:
            prev = curr
            curr = curr.next
        if prev is None:
            self.head = curr.next
        elif curr:
            prev.next = curr.next
            curr.next = None

    # Método para obtener el ultimo nodo
    def get_last_node(self):
        temp = self.head
        while(temp.next is not None):
            temp = temp.next
        return temp.data

    # Método para imprimir la lista de nodos
    def print_list( self ):
        node = self.head
        while node != None:
            print(node.data +" => ", flush= True, end ="")
            node = node.next